In [1]:
import pandas as pd
import numpy as np
import geonamescache
import pycountry
import difflib
import warnings
import statistics as stat
import re

from countryinfo import CountryInfo
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

In [2]:
df = pd.read_csv("../data/4_shark_geography.csv")

In [3]:
df.head()

,Case Number,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,Final_year,Final_month,Final_day
0,Attack-6286,Boating,United States,New Caledonia,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0,2018,6,25
1,Attack-6285,Unprovoked,United States,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0,2018,6,18
2,Attack-6284,Invalid,United States,Haiti,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0,2018,6,9
3,Attack-6283,Unprovoked,Australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0,2018,6,8
4,Attack-6282,Provoked,Mexico,Colombia,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0,2018,6,4


# Limpieza columna "Activity"

In [4]:
def activity(x):
    try:
        x = x.lower()
        if "fis" in x:
            return "Fishing"
        elif "sta" in x:
            return "Standing"
        elif "pad" in x:
            return "Padding"
        elif "sur" in x:
            return "Surfing"
        elif "swi" in x:
            return "Swimming"
        elif "bat" in x:
            return "Bathing"
        elif "div" in x:
            return "Diving"
        elif "sno" in x:
            return "Snorkel"
        elif "boa" in x:
            return "boarding"
        else:
            return "Other"
    except:
        return "Other"

In [5]:
len(df[df.Activity.isnull()])

535

In [6]:
df.Activity = df.Activity.apply(activity)

In [7]:
df.Activity.value_counts()

Other       1411
Surfing     1229
Fishing     1223
Swimming    1081
Diving       503
boarding     314
Bathing      187
Standing     154
Snorkel       90
Padding       40
Name: Activity, dtype: int64

In [8]:
len(df[df.Activity.isnull()])

0

Con la función utilizada simplificamos la columna en 10 categorías. Previamente a la aplicación de la función contamos con 500 nulos que al no tener información sobre la actividad introducimos en la categoría "Otros", la cual resulta ser la más amplia. De cerca le siguen las categorias de pesca y surf con valores muy similares.

# limpieza columna "Name"

A continuación limpiarémos la columna name. Revisando a través de value_counts() detectamos varios valores como male or boy, los cuales nos sirven para verificar la columna de género. Se procede a la utilización y limpieza de estos valores. A continuación, al ser útiles en la columna de nombre los sustituirémos por Unknown, esto serña tras la verificación del género.

In [9]:
def name_1 (x):
    try:
        x = x.lower()
        if "men" in x:
            return "M"
        if "male" in x:
            return "M"
        elif "female" in x:
            return "F"
        elif "girl" in x:
            return "F"
        elif "woman" in x:
            return "F"
        elif "boy" in x:
            return "M"
        elif "man" in x:
            return "M"
        else:
            return x
    except:
        return x
    
def name_2(x):
    if x == "boat":
        return "Unknown"
    elif x == "child":
        return "Unknown"
    elif x== "anonymous":
        return "Unknown"
    elif x == "sailor":
        return "Unknown"
    elif x == "unidentified":
        return "Unknown"
    elif x == "a sailor":
        return "Unknown"
    elif x == "a native":
        return "Unknown"
    elif x == "a soldier":
        return "Unknown"
    elif x == "a pearl diver":
        return "Unknown"
    else:
        return x
    
def name_3(x):
    if x == "M":
        return "Unknown male"
    elif x == "F":
        return "Unknown female"
    elif x == "unknown":
        return "Unknown"
    else:
        return x

In [10]:
df.Name = df.Name.apply(name_1)
df.Name = df.Name.apply(name_2)

Ahora que tenemos los nombres agrupados procedemos a utilizar las categorias creadas "M" y "F" para verificar y rellenar la columna género.

In [11]:
df["Sex"] = df['Sex ']
df["Species"] = df['Species ']
df.drop(["Sex ", "Species "], axis=1, inplace=True)

In [12]:
df.insert(7, 'Sex', df.pop("Sex"))

Vemos 190 registros en los que Name "M"/"F" y Sex son diferentes. Vamos a priorizar el registro dado en la columna Sex, sin embargo tb vemos algunos valores NaN que rellenaremos con el valor obtenido de la exploración previa.

In [13]:
remplazo = df[(df['Name'] == "M") & (df['Sex'] != "M") & df['Sex'].isna()]
df.loc[remplazo.index, 'Sex'] = remplazo['Name']

Para terminar con la columna de nombre vamos a substituir los valores creados M y F por "Unknown", a esta categoría también añadiremos los Nan

In [14]:
df.Name = df.Name.apply(name_3)
df['Name'] = df['Name'].fillna("Unknown")

# Limpieza columna "Sex"

Al ya haber concatenado la columna Sex con los registros obtenidos de la columna Name en este paso únicamente rellenaremos los NaN Sex con valores Unknown. También limpiaremos las categorias desconocidas.

In [15]:
def genero(x):
    if x == "M ":
        return "M"
    elif x == "lli":
        return "M"
    elif x == "N":
        return "Unknown"
    elif x == ".":
        return "Unknown"
    else:
        return x

In [16]:
df.Sex = df.Sex.fillna("Unknown")
df.Sex = df.Sex.apply(genero)

In [17]:
df.Sex.value_counts()

M          5084
F           633
Unknown     515
Name: Sex, dtype: int64

# Limpieza columna "Age"

In [18]:
def age_teen(x):
    try:
        x = x.lower()
        if "teen" in x:
            return 18
        else: return x
    except:
        return x

In [19]:
def age_several(x):
    try:
        nums = re.findall(r'\b\d{2}\b', x)
        nums = [int(num) for num in nums]
        if len(nums) > 0:
            return stat.mean(nums)
        return x
    except: return x

In [20]:
def age_letter(x):
    try:
        x = int(x)
        # Use a regular expression to capture numeric parts
        match = re.search(r'(\d+(\.\d+)?)', x)

        if match:
            num = match.group(1)
            num = re.sub(r'[^\d.]', '', num)  # Remove non-numeric characters except for decimal points
            if "." in num:
                num = int(float(num))  # Convert to float and round to an integer
            return num
        else:
            return x
    except:
        return x

In [40]:
def rellenar(x):
    if type(x) == int:
        return x
    else:
        return 0

In [22]:
df.Age = df.Age.apply(age_teen)
df.Age = df.Age.apply(age_several)
df.Age = df.Age.apply(age_letter)
df.Age = df.Age.apply(rellenar)
df['Age'].fillna(0, inplace=True)  # rellenamos los valores nulos con 0

Para limpiar la columna age se decide recuperar las edades de los strings y convertir la columna a tipo int. Tanto los valores nulos cómo los no deseados se convierten a 0. A la hora de realizar un análisis estadístico de la columna será necesario tener en cuenta que los valores 0 no deben influenciar los resultados.

# Limpieza columna "Injury"

En un primer vistazo a los datos aportados por la columna "Injury" observamos un considerable número de registros catalogados como "FATAL". Decidimos utilizar estos para contrastar los datos aportados por la siguiente columna Fatal (Y/N)

In [23]:
df.Injury.value_counts()[:5]

FATAL          779
Survived        94
Foot bitten     87
No injury       81
Leg bitten      72
Name: Injury, dtype: int64

In [24]:
len(df[(df['Injury'] == 'FATAL') & (df['Fatal (Y/N)'] == 'Y')])

760

Observamos que hay una diferencia entre el número de registros "FATAL" y el número de coincidencias con la columna fatal(Y/N). A continuación cambiaremos el nombre de la columna fatal para una mayor comodidad. También decidimos que allí donde se haya indicado que la herida fue fatal situaremos el mismo registro en la columna "Deadly"

In [25]:
df["Deadly"] = df['Fatal (Y/N)']
df.drop(['Fatal (Y/N)'],axis=1,inplace=True)

In [26]:
df.loc[df['Injury'] == 'FATAL', 'Deadly'] = 'Y'

In [27]:
len(df[(df['Injury'] == 'FATAL') & (df['Deadly'] == 'Y')])

779

Vemos que en la columna Injury tenemos 26 nulos. Categorizaremos estos registros como "Unknown". Para finar el tratamiento de estos datos aplicaremos formulas para corregir facilitar la lectura.

In [28]:
len(df[df.Injury.isnull()])

26

In [29]:
df.Injury.fillna("Unknown",inplace=True)

In [30]:
df.Injury = df.Injury.apply(lambda s: s.lower())
df.Injury = df.Injury.apply(lambda s: s.capitalize())

# Limpieza columna "Deadly"

A continuación limpiamos los valores incongruentes de la columna Deadly categorizandolos como "Unknown".

In [31]:
def dead(x):
    try:
        if x == "N":
            return x
        elif x == "Y":
            return x
        else:
            return "Unknown"
    except:
        return x

In [32]:
df.Deadly = df.Deadly.apply(dead)

In [33]:
df.Deadly.value_counts()

N          4261
Y          1377
Unknown     594
Name: Deadly, dtype: int64

# Limpieza columna "href"

Para la limpieza de las columnas "href" y "href formula" hemos comprobado que únicamente existe un valor nulo en la columna "href formula". También se ha comprobado que aunque aparentemente parezcan contener la misma información existen 60 registros donde estas columnas son diferentes. Debido a la inexistencia de nulos en la columna "href" se decide confiar en esta y eliminar "href formula", ya que no tendría sentido tener dos columnas quee tuvieran la misma información.

In [34]:
len(df[df.href.isnull()])

0

In [35]:
len(df[df["href formula"].isnull()])

1

In [36]:
len(df[df.href != df["href formula"]])

60

In [37]:
df.drop(["href formula"],axis=1,inplace=True)

In [39]:
df.to_csv("../data/5_shark_person.csv",index=False)